<a href="https://colab.research.google.com/github/SelenaNahra/DL/blob/main/HW5P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 20
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 1024
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded.permute(1, 0, 2))  # Permute for Transformer
        output = self.fc(transformer_output[-1, :, :])  # Get the output of the last Transformer block
        return output

# Hyperparameters
hidden_size = 128
num_layers = 2
nhead = 1
learning_rate = 0.001
epochs = 100

# Model, loss, and optimizer
model = CharTransformer(len(chars), hidden_size, len(chars), num_layers, nhead).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            val_loss += criterion(outputs, targets).item()

    end_time = time.time()
    training_time = end_time - start_time

    # Print statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_val_loss = val_loss / len(test_loader)
    val_accuracy = correct / total
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}, Validation Loss: {epoch_val_loss}, Validation Accuracy: {val_accuracy}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_int[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Counting model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time:{training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
total_complexity = 0
for p in model.parameters():
    total_complexity += torch.prod(torch.tensor(p.shape))
print(f"Total trainable parameters in the model: {total_complexity}")

Epoch 10, Loss: 2.264006566017046, Validation Loss: 2.259694847491903, Validation Accuracy: 0.3242270328911682
Epoch 20, Loss: 2.212965158694381, Validation Loss: 2.2393525335766853, Validation Accuracy: 0.32962446597241213
Epoch 30, Loss: 2.1758991909136465, Validation Loss: 2.2376779733447854, Validation Accuracy: 0.3313772868484639
Epoch 40, Loss: 2.1462161822603383, Validation Loss: 2.245298114391642, Validation Accuracy: 0.33234559710224193
Epoch 50, Loss: 2.1185944769360603, Validation Loss: 2.249302469262289, Validation Accuracy: 0.3314310818625627
Epoch 60, Loss: 2.0952320824795905, Validation Loss: 2.263546364023051, Validation Accuracy: 0.330099655263618
Epoch 70, Loss: 2.07433977244644, Validation Loss: 2.270742569494685, Validation Accuracy: 0.3296782609865109
Epoch 80, Loss: 2.0557687468485, Validation Loss: 2.282223565862813, Validation Accuracy: 0.3286068436223769
Epoch 90, Loss: 2.039859058386689, Validation Loss: 2.2909564840684244, Validation Accuracy: 0.3276788796291

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 30
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded.permute(1, 0, 2))  # Permute for Transformer
        output = self.fc(transformer_output[-1, :, :])  # Get the output of the last Transformer block
        return output

# Hyperparameters
hidden_size = 256
num_layers = 3
nhead = 2
learning_rate = 0.0001
epochs = 100

# Model, loss, and optimizer
model = CharTransformer(len(chars), hidden_size, len(chars), num_layers, nhead).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            val_loss += criterion(outputs, targets).item()

    end_time = time.time()
    training_time = end_time - start_time

    # Print statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_val_loss = val_loss / len(test_loader)
    val_accuracy = correct / total
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}, Validation Loss: {epoch_val_loss}, Validation Accuracy: {val_accuracy}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_int[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Counting model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time:{training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
total_complexity = 0
for p in model.parameters():
    total_complexity += torch.prod(torch.tensor(p.shape))
print(f"Total trainable parameters in the model: {total_complexity}")

Epoch 10, Loss: 2.1567983181908836, Validation Loss: 2.1744311128754212, Validation Accuracy: 0.34867061455218695
Epoch 20, Loss: 2.0690518340979644, Validation Loss: 2.17501256374265, Validation Accuracy: 0.3515710103867344
Epoch 30, Loss: 1.9843201707371063, Validation Loss: 2.213150710123952, Validation Accuracy: 0.34721369237872807
Epoch 40, Loss: 1.9071941861323904, Validation Loss: 2.259049965811405, Validation Accuracy: 0.3432418983920062
Epoch 50, Loss: 1.8397052513923853, Validation Loss: 2.3113317740626944, Validation Accuracy: 0.33712730810093555
Epoch 60, Loss: 1.7814675004606746, Validation Loss: 2.373403570601397, Validation Accuracy: 0.3317434203153228
Epoch 70, Loss: 1.731665183758503, Validation Loss: 2.422033537890948, Validation Accuracy: 0.32880267894366416
Epoch 80, Loss: 1.6871482045980137, Validation Loss: 2.464687901533546, Validation Accuracy: 0.3265433288654387
Epoch 90, Loss: 1.6500292498697444, Validation Loss: 2.496457652531338, Validation Accuracy: 0.32367

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 50
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 512
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded.permute(1, 0, 2))  # Permute for Transformer
        output = self.fc(transformer_output[-1, :, :])  # Get the output of the last Transformer block
        return output

# Hyperparameters
hidden_size = 128
num_layers = 3
nhead = 2
learning_rate = 0.0001
epochs = 100

# Model, loss, and optimizer
model = CharTransformer(len(chars), hidden_size, len(chars), num_layers, nhead).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to GPU if available
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            val_loss += criterion(outputs, targets).item()

    end_time = time.time()
    training_time = end_time - start_time

    # Print statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_val_loss = val_loss / len(test_loader)
    val_accuracy = correct / total
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}, Validation Loss: {epoch_val_loss}, Validation Accuracy: {val_accuracy}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_int[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Counting model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time:{training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
total_complexity = 0
for p in model.parameters():
    total_complexity += torch.prod(torch.tensor(p.shape))
print(f"Total trainable parameters in the model: {total_complexity}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10, Loss: 2.291786092762556, Validation Loss: 2.2778045644453906, Validation Accuracy: 0.3209365711954597
Epoch 20, Loss: 2.240403465067458, Validation Loss: 2.241381116416476, Validation Accuracy: 0.3299337873034801
Epoch 30, Loss: 2.2066812148671415, Validation Loss: 2.230577992189915, Validation Accuracy: 0.33186144197535294
Epoch 40, Loss: 2.1773063008017877, Validation Loss: 2.2272107590229138, Validation Accuracy: 0.3329507910108531
Epoch 50, Loss: 2.149008478026792, Validation Loss: 2.2334770303253735, Validation Accuracy: 0.33253387965158765
Epoch 60, Loss: 2.1218119856275774, Validation Loss: 2.2412202768369553, Validation Accuracy: 0.33061967373323947
Epoch 70, Loss: 2.09594748441474, Validation Loss: 2.2490865484290166, Validation Accuracy: 0.3300234456603114
Epoch 80, Loss: 2.07061270421357, Validation Loss: 2.263133306568916, Validation Accuracy: 0.3275130116690351
Epoch 90, Loss: 2.0486597609260064, Validation Loss: 2.2773393715193513, Validation Accuracy: 0.3257691